In [6]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()

In [7]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()
x = torch.randn(3, 3, requires_grad=False, device=device)
y = torch.randn(3, 3, requires_grad=False, device=device)
torch_xla.sync()
out = torch.einsum('...n,mn->...m', x, y)
print(torch_xla._XLAC._get_xla_tensors_text([out]))

Einsum: Call native function
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

Einsum: Call tensor method call
Einsum: Build lowered eisum


In [3]:
import torch
import torch_xla
import torch_xla.runtime
device = torch_xla.device()
x = torch.randn(3, 3, requires_grad=True, device=device)
y = torch.randn(3, 3, requires_grad=True, device=device)
torch_xla.sync()
out = torch.einsum('...n,mn->...m', x, y)
print(torch_xla._XLAC._get_xla_tensors_text([out]))

Einsum: Calling lowered eisum
Einsum: Build lowered eisum
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}



In [6]:
print("-- Does creating with gradient enabled make a difference?")

print("-- Test1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2")
x_2= torch.randn(3, 3, requires_grad=True)
y_2= torch.randn(3, 3, requires_grad=True)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test3")
x_1= torch.randn(3, 3, requires_grad=False, device=device)
y_1= torch.randn(3, 3, requires_grad=False, device=device)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test4")
x_2= torch.randn(3, 3, requires_grad=False, device=device)
y_2= torch.randn(3, 3, requires_grad=False, device=device)
with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

-- Does creating with gradient enabled make a difference?
-- Test1
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Test2
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Test3
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = s64[] xla::device_data(), xla_shape=s64[]
  %1 = s64[] prim::Constant(), xla_shape=s64[]
  %2 = s64[] aten::mul(%1, %0), xla_shape=s64[]
  %3 = s64[] prim::Constant(), xla_shape=s64[]
  %4 = s64[] aten::add(%3,

It doesn't seem like gradient made a difference. However, device creation mattered

In [5]:
print("-- Is there a different if we create in the CPU, and then transfer to XLA?")
print("-- Check 1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 2")
x_1= torch.randn(3, 3, requires_grad=True).to('xla')
y_1= torch.randn(3, 3, requires_grad=True).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 3")
with torch_xla.runtime.xla_device():
  x_1= torch.randn(3, 3, requires_grad=False).to('xla')
  y_1= torch.randn(3, 3, requires_grad=False).to('xla')
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 4")
with torch_xla.runtime.xla_device():
  x_1= torch.randn(3, 3, requires_grad=True).to('xla')
  y_1= torch.randn(3, 3, requires_grad=True).to('xla')
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Check 5")
x_1= torch.randn(3, 3, requires_grad=True)
y_1= torch.randn(3, 3, requires_grad=True)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1.to('xla'), y_1.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))



-- Is there a different if we create in the CPU, and then transfer to XLA
-- Check 1
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Check 2
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %1 = f32[3,3]{1,0} xla::device_data(), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3]{1,0} aten::einsum(%1, %0), xla_shape=f32[3,3]{1,0}, ROOT=0
}

-- Check 3
Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
IR {
  %0 = s64[] xla::device_data(), xla_shape=s64[]
  %1 = s64[] prim::Constant(), xla_shape=s64[]
  %2 = s64[] aten::mul(%1, %0), xla_shape=s64[]
  %3 = s64[] prim::Constant(), xla_shape=s64[]
  %4

Conclusion:
It seems like creating the tensor in the xla_device is causing the failure. If we create it, and then transition to the TPU, it succeeds

In [5]:
print("-- Does creating with gradient enabled make a difference?")

print("-- Test1")
x_1= torch.randn(3, 3, requires_grad=False).to('xla')
y_1= torch.randn(3, 3, requires_grad=False).to('xla')
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test2")
x_2= torch.randn(3, 3, requires_grad=True)
y_2= torch.randn(3, 3, requires_grad=True)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test3")
x_1= torch.randn(3, 3, requires_grad=False, device=device)
y_1= torch.randn(3, 3, requires_grad=False, device=device)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_1, y_1)
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

print("-- Test4")
x_2= torch.randn(3, 3, requires_grad=False, device=device)
y_2= torch.randn(3, 3, requires_grad=False, device=device)
with torch_xla.runtime.xla_device():
  out_einsum = torch.einsum('...m,mn->...n', x_2.to('xla'), y_2.to('xla'))
print(torch_xla._XLAC._get_xla_tensors_text([out_einsum]))

-- Does creating with gradient enabled make a difference?
-- Test1


RuntimeError: PyTorch is not linked with support for xla devices

In [1]:
import torch
from torch import Tensor
from torch.library import custom_op
import time
import torch_xla
import torch_xla.runtime
device = torch_xla.device()
%env TORCH_SHOW_DISPATCH_TRACE=1

@custom_op("xla::custom_linear_forward123", schema="(Tensor input, Tensor weight) -> Tensor", mutates_args=())
def custom_linear_forward123(input: Tensor, weight: Tensor):
    return torch.einsum('...n,mn->...m', input, weight)

X = torch.zeros(3, 3, requires_grad=False, device='xla')
Y = torch.zeros(3, 3, requires_grad=False, device='xla')

def test_lowering(func):
  time.sleep(1)
  out = func(X, Y)
  time.sleep(1)
  ir = torch_xla._XLAC._get_xla_tensors_text([out])
  if 'einsum' not in ir:
    print("!!!!!!!!!!WRONG!!!!!!!!!!! Did not find einsum in lowering")
    print("IR:")
    print(ir)
  else:
    print("OK")

env: TORCH_SHOW_DISPATCH_TRACE=1


In [2]:
@custom_op("xla::custom_max_pool", schema="(Tensor input) -> Tensor", mutates_args=())
def custom_max_pool(input: Tensor):
    return torch.max_pool2d(input)

In [3]:
test_lowering(lambda a, b: custom_linear_forward123(a, b))

Einsum: Call native function
Einsum: Call tensor method call
Einsum: Build lowered eisum
OK


In [4]:
test_lowering(lambda a, b: torch.einsum('...n,mn->...m', a, b))

!!!!!!!!!!WRONG!!!!!!!!!!! Did not find einsum in lowering
IR:
IR {
  %0 = f32[] prim::Constant(), xla_shape=f32[]
  %1 = f32[3,3]{1,0} aten::expand(%0), xla_shape=f32[3,3]{1,0}
  %2 = f32[3,3,1]{2,1,0} aten::view(%1), xla_shape=f32[3,3,1]{2,1,0}
  %3 = f32[1,3,3]{0,2,1} aten::permute(%2), xla_shape=f32[1,3,3]{0,2,1}
  %4 = f32[3,3,1]{2,0,1} aten::permute(%3), xla_shape=f32[3,3,1]{2,0,1}
  %5 = f32[1,3,3]{2,1,0} aten::view(%4), xla_shape=f32[1,3,3]{2,1,0}
  %6 = f32[] prim::Constant(), xla_shape=f32[]
  %7 = f32[3,3]{1,0} aten::expand(%6), xla_shape=f32[3,3]{1,0}
  %8 = f32[3,3,1]{2,1,0} aten::view(%7), xla_shape=f32[3,3,1]{2,1,0}
  %9 = f32[3,1,3]{1,2,0} aten::permute(%8), xla_shape=f32[3,1,3]{1,2,0}
  %10 = f32[3,3,1]{2,1,0} aten::permute(%9), xla_shape=f32[3,3,1]{2,1,0}
  %11 = f32[1,3,3]{2,1,0} aten::view(%10), xla_shape=f32[1,3,3]{2,1,0}
  %12 = f32[1,3,3]{2,1,0} aten::matmul(%11, %5), xla_shape=f32[1,3,3]{2,1,0}
  %13 = f32[3,1,3]{2,1,0} aten::view(%12), xla_shape=f32[3,1,3]{2,1,